# Interpolacja Hermite'a i Lagrange'a 
## Porównanie metod: Hermite (Newton DD + pochodna), Lagrange (wzór standardowy), Lagrange (Newton DD)

In [2]:
import numpy as np
import pandas as pd
import os
import shutil
import matplotlib.pyplot as plt
from typing import Callable, Literal 
import ipympl
%matplotlib ipympl

## Generowanie wartości x (Węzłów)

In [3]:
def generate_chebyshev_roots(n: int, a: np.float64, b: np.float64):
  # adjust n to account for the two additional nodes at the ends
  n_adjusted = n - 2
  chebyshev_roots = np.array([(a + b) / 2 + (b - a) / 2 * np.cos((2 * k + 1) / (2 * n_adjusted) * np.pi) for k in range(n_adjusted)], dtype=np.float64)
  # add the endpoints of the interval
  return np.concatenate(([a], chebyshev_roots[::-1], [b]))

def generate_function_uniform_nodes(n: int, interval: tuple[np.float64, np.float64], func):
  a,b= interval[0], interval[1]
  xs = np.linspace(a,b,n)
  tups = []
  for x in xs:
    tups.append((x,func(x)))
  return np.array(tups)

def generate_function_chebyshev_nodes(n: int, interval: tuple[np.float64, np.float64], func):
  a,b= interval[0], interval[1]
  xs = generate_chebyshev_roots(n,a,b)
  tups = []
  for x in xs:
    tups.append((x,func(x)))
  return np.array(tups)


## Interpolacja Hermite'a (Newton DD + Pierwsza Pochodna)

In [4]:
def hermite_divided_difference_table(nodes: np.ndarray, deriv_nodes: np.ndarray) -> np.ndarray:

  n = len(nodes)
  nodes_x = nodes[:,0]
  nodes_y = nodes[:,1]
  nodes_derivative_y = deriv_nodes[:,1]
  z = np.zeros(2 * n, dtype=np.float64) # Duplicate values for Hermite interpolation
  dp = np.zeros((2 * n, 2 * n), dtype=np.float64)  # Divided difference table

  # Fill z with duplicated x values
  for i in range(n):
    z[2 * i] = nodes_x[i]
    z[2 * i + 1] = nodes_x[i]

  # Fill in function values
  for i in range(n):
    dp[2 * i, 0] = nodes_y[i]
    dp[2 * i + 1, 0] = nodes_y[i]
    dp[2 * i + 1, 1] = nodes_derivative_y[i]  # First derivative constraint
    if i != 0:
      dp[2 * i, 1] = (dp[2 * i, 0] - dp[2 * i - 1, 0]) / (z[2 * i] - z[2 * i - 1])


  for i in range(2,2*n):
    for j in range(2,i+1):
      dp[i,j] = (dp[i,j-1] - dp[i-1,j-1])/(z[i] - z[i-j])

  return z, dp


def get_hermite_interpolation_func(nodes: np.ndarray, deriv_nodes: np.ndarray) -> Callable[[float], float]:
    
    nodes_x, dp = hermite_divided_difference_table(nodes, deriv_nodes)
    
    def func_hermite(x: float) -> float:
      # n = len(nodes_x)
      # result = dp[0,n-1]
      # for i in range(n-2,-1,-1):
      #   result = result * (x-nodes_x[i]) + dp[0,i]
      # return result
      
      n = len(nodes_x)
      result = dp[n - 1, n - 1]
      for i in range(n - 2, -1, -1):
        result = result * (x - nodes_x[i]) + dp[i, i]
      return result
      
      # result = dp[0, 0]  # a0
      # product_term = 1.0
      # for i in range(1, len(nodes_x)):
      #   product_term *= (x - nodes_x[i - 1])
      #   result += dp[i, i] * product_term
      # return result

    
    return func_hermite

## Interpolacja Lagrange'a (Wzór Standardowy)

In [5]:
def get_lagrange_interpolation_func(nodes: np.ndarray[(np.float64, np.float64)]) -> Callable[[np.float64], np.float64]:
  n = len(nodes)
  x_nodes = nodes[:, 0]
  y_nodes = nodes[:, 1]
  def coefficient(k):
    return np.prod([1/(x_nodes[k] - x_nodes[i]) for i in range(n) if i != k])
  coefficients = np.array([coefficient(k) for k in range(n)])
  
  def polymial_term(x,k):
    return np.prod([(x - x_nodes[i]) for i in range(n) if i!=k])
  
  return lambda x: np.sum(y_nodes * coefficients * np.array([polymial_term(x,k) for k in range(n)]))

## Interpolacja Lagrange'a (Newton DD)

In [6]:
def divided_difference_table(nodes: np.ndarray[(np.float64, np.float64)]) -> np.ndarray:
  n = len(nodes)
  nodes_x = nodes[:,0]
  nodes_y = nodes[:,1]
  dp = np.ndarray((n,n), dtype=np.longdouble)
  dp[:,0] = nodes_y
  for i in range(1,n):
    for j in range(1,i+1):
      dp[i,j] = (dp[i,j-1] - dp[i-1,j-1])/(nodes_x[i] - nodes_x[i-j])
  
  return dp

def get_newton_interpolation_func(nodes: np.ndarray[(np.float64, np.float64)]) -> Callable[[np.float64], np.float64]:
  n = len(nodes)
  nodes_x = nodes[:,0]
  dp = divided_difference_table(nodes)
  
  def func(x):
    result = dp[n-1,n-1]
    for i in range(n-2,-1,-1):
      result = result * (x-nodes_x[i]) + dp[i,i]
    return result

  return func
  

## Błąd Pomiarowy

In [7]:
NUMBER_OF_PROBES = 1000

def get_max_error(func, interpolation_func, interval):
  xs = np.linspace(interval[0], interval[1], NUMBER_OF_PROBES)
  return max([np.abs(func(x) - interpolation_func(x)) for x in xs])

def get_squared_error(func, interpolation_func, interval):
  xs = np.linspace(interval[0], interval[1], NUMBER_OF_PROBES)
  return np.sqrt(sum([(func(x) - interpolation_func(x))**2 for x in xs])) / NUMBER_OF_PROBES

## Wizualizacja

In [8]:
def plot_function(
    x_values: np.ndarray[np.float64], 
    func: Callable[[np.float64], np.float64], 
    nodes: np.ndarray | None = None,
    x_lim: tuple[float, float] | None = None,
    y_lim: tuple[float, float] | None = None,
    x_scale: str = 'linear',
    y_scale: str = 'linear',
    function_name: str = 'Funkcja bazowa',
    nodes_name: str = 'Węzły interpolacji',
    color: str | None = None,        # Added color parameter
    linestyle: str = '-'             # Added linestyle parameter
):
    # Evaluate the function, handle potential NaNs from interpolation
    y_values = np.array([func(x) for x in x_values])
    valid_indices = ~np.isnan(y_values) # Find where y is not NaN
    
    # Plot only valid points
    if color is not None:
        plt.plot(x_values[valid_indices], y_values[valid_indices], label=function_name, color=color, linestyle=linestyle)
    else:
        plt.plot(x_values[valid_indices], y_values[valid_indices], label=function_name, linestyle=linestyle)
        
    if nodes is not None and len(nodes) > 0:
        plt.scatter(nodes[:, 0], nodes[:, 1], color='red', label=nodes_name, zorder=5) # zorder to plot on top
    
    plt.xscale(x_scale)
    plt.yscale(y_scale)
    
    if x_lim is not None:
        plt.xlim(x_lim)
    if y_lim is not None:
        plt.ylim(y_lim)
        
    # Adding grid for better readability
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
        
    plt.xlabel('x')
    plt.ylabel('y')
    # plt.legend() # Legend is handled outside in the main loop for combined plots
    plt.title('Wizualizacja funkcji') # Generic title, will be overridden

## Definicja Funkcji Bazowej i Pochodnej

In [9]:
m = 5
k = 0.5
interval = (-5.0, 5.0) # Use floats for interval
# N will be defined in the loop below

func = lambda x: x**2 - m*np.cos((np.pi * x) / k)
# Define first derivative for Hermite
func_derivative = lambda x: 2*x + ((m * np.pi) / k) * np.sin((np.pi * x) / k)
# Define second derivative (needed for some spline methods, though not used in this version)
func_second_deriv = lambda x: 2 + ((m * np.pi**2) / k**2) * np.cos((np.pi * x) / k)

## Przykładowe Użycie (dla pojedynczej liczby węzłów - można odkomentować do testów)

In [10]:
# N_test = 11 # Example number of nodes

# # Generate nodes
# nodes_linear = generate_function_uniform_nodes(N_test, interval, func)
# nodes_derivative_linear = generate_function_uniform_nodes(N_test, interval, func_derivative)
# nodes_chebyshev = generate_function_chebyshev_nodes(N_test, interval, func)
# nodes_derivative_chebyshev = generate_function_chebyshev_nodes(N_test, interval, func_derivative)

# # Hermite
# hermite_linear = get_hermite_interpolation_func(nodes_linear, nodes_derivative_linear)
# hermite_chebyshev = get_hermite_interpolation_func(nodes_chebyshev, nodes_derivative_chebyshev)

# # Lagrange Standard
# lagrange_std_linear = get_lagrange_standard_interpolation_func(nodes_linear)
# lagrange_std_chebyshev = get_lagrange_standard_interpolation_func(nodes_chebyshev)

# # Lagrange Newton
# lagrange_newton_linear = get_lagrange_newton_interpolation_func(nodes_linear)
# lagrange_newton_chebyshev = get_lagrange_newton_interpolation_func(nodes_chebyshev)

# # Example Plot (Linear Nodes)
# plt.figure(figsize=(10, 6))
# plot_x_test = np.linspace(interval[0], interval[1], 500)
# plot_function(plot_x_test, func, None, y_lim=(-10, 40), function_name='Funkcja Oryginalna', color='black')
# plot_function(plot_x_test, hermite_linear, nodes_linear, y_lim=(-10, 40), function_name='Hermite (Linear)', nodes_name='Węzły Równoodległe', color='green')
# plot_function(plot_x_test, lagrange_std_linear, None, y_lim=(-10, 40), function_name='Lagrange Std (Linear)', color='blue', linestyle='--')
# plot_function(plot_x_test, lagrange_newton_linear, None, y_lim=(-10, 40), function_name='Lagrange Newton (Linear)', color='purple', linestyle='-.')
# plt.title(f'Interpolacje dla N={N_test} węzłów równoodległych')
# plt.legend()
# plt.grid(True)
# plt.show()


## Pętla Główna - Generowanie Wykresów i Danych dla Różnych Liczb Węzłów

In [11]:
# Updated Cell 9: Main Processing Loop for Comparison

# --- Configuration ---
PATH_TO_SAVE_MAIN = os.path.join('.', 'img_comparison')
PATH_TO_SAVE_DATA = os.path.join('.', 'data_comparison')
INTERP_METHODS = {
    'hermite': {
        'func': get_hermite_interpolation_func,
        'name': "Zagadnienie Hermite'a (Wzór Newtona)",
        'color': 'green',
        'linestyle': '-'
    },
    'lagrange_standard': {
        'func': get_lagrange_interpolation_func,
        'name': "Zagadnienie Lagrange'a (Wzór Lagrange'a)",
        'color': 'blue',
        'linestyle': '--'
    },
    'lagrange_newton': {
        'func': get_newton_interpolation_func,
        'name': "Zagadnienie Lagrange'a (Wzór Newtona)",
        'color': 'purple',
        'linestyle': '-.'
    }
}
NODE_TYPES = {
    'linear': {
        'func': generate_function_uniform_nodes,
        'name': "Równoodległe",
        'nodes_name': 'Węzły Równoodległe'
    },
    'chebyshev': {
        'func': generate_function_chebyshev_nodes,
        'name': "Czebyszewa",
        'nodes_name': 'Węzły Czebyszewa'
    }
}

# --- Paths and Directories ---
paths = {}
for method_key in INTERP_METHODS.keys():
    paths[method_key] = {}
    for node_key in NODE_TYPES.keys():
        paths[method_key][node_key] = os.path.join(PATH_TO_SAVE_MAIN, method_key, node_key)
paths['combined'] = os.path.join(PATH_TO_SAVE_MAIN, 'combined')


# Cleanup existing directories
if os.path.exists(PATH_TO_SAVE_MAIN):
    shutil.rmtree(PATH_TO_SAVE_MAIN)
# Don't delete data dir if it might contain other useful files
# if os.path.exists(PATH_TO_SAVE_DATA):
#     shutil.rmtree(PATH_TO_SAVE_DATA)

os.makedirs(PATH_TO_SAVE_DATA, exist_ok=True)
os.makedirs(paths['combined'], exist_ok=True)
for method_key in INTERP_METHODS.keys():
    for node_key in NODE_TYPES.keys():
        os.makedirs(paths[method_key][node_key], exist_ok=True)

# --- Initialization ---
data_records = []
N_max = 51  # Max number of nodes (adjust as needed)
# Generate x-values for plotting
plot_x = np.linspace(interval[0], interval[1], NUMBER_OF_PROBES) # Use NUMBER_OF_PROBES for consistency with error calc
y_lim_plot = (-10, 40)
x_lim_plot = (interval[0] - 1, interval[1] + 1) # Add padding

N_SET = [11,21]

# --- Main Loop ---
# Start from N=1 for Hermite, N=2 for Lagrange
for i in N_SET:
    print(f"Processing n={i} nodes...")
    
    # Generate nodes (Lagrange only needs (x,y))
    nodes_all = {}
    nodes_deriv_all = {}
    for node_key, node_info in NODE_TYPES.items():
        nodes_all[node_key] = node_info['func'](i, interval, func)
        # Generate derivatives needed only for Hermite
        nodes_deriv_all[node_key] = node_info['func'](i, interval, func_derivative)

    # --- Combined Plots Initialization ---
    fig_comb_linear = plt.figure(figsize=(12, 7))
    plt.plot(plot_x, [func(x) for x in plot_x], label='Funkcja Oryginalna', linestyle=':', color='black', alpha=0.7)
    plt.ylim(y_lim_plot)
    plt.xlim(x_lim_plot)
    plt.grid(True)
    
    fig_comb_chebyshev = plt.figure(figsize=(12, 7))
    plt.plot(plot_x, [func(x) for x in plot_x], label='Funkcja Oryginalna', linestyle=':', color='black', alpha=0.7)
    plt.ylim(y_lim_plot)
    plt.xlim(x_lim_plot)
    plt.grid(True)

    combined_figs = {'linear': fig_comb_linear, 'chebyshev': fig_comb_chebyshev}

    # --- Interpolation, Error Calculation, and Plotting ---
    for method_key, method_info in INTERP_METHODS.items():
        for node_key, node_info in NODE_TYPES.items():
            current_nodes = nodes_all[node_key]

            # Skip Lagrange for N=1
            if i < 2 and method_key != 'hermite':
                error_max = np.inf
                error_scaled_rmse = np.inf
                interp_func = lambda x: np.nan # Dummy func
                
            else:
                try:
                    # Calculate interpolation function
                    if method_key == 'hermite':
                        # Hermite needs nodes AND derivatives
                        interp_func = method_info['func'](current_nodes, nodes_deriv_all[node_key])
                    else:
                        # Lagrange methods only need nodes (x,y)
                        interp_func = method_info['func'](current_nodes)

                    # Calculate errors
                    error_max = get_max_error(func, interp_func, interval)
                    error_scaled_rmse = get_squared_error(func, interp_func, interval)

                except Exception as e:
                    print(f"  ERROR calculating {method_info['name']} with {node_info['name']} nodes (n={i}): {e}")
                    error_max = np.inf
                    error_scaled_rmse = np.inf
                    # Create a dummy function returning NaN to avoid plotting errors
                    interp_func = lambda x: np.nan

            # --- Individual Plot --- (Only plot if calculation was attempted)
            if not (i < 2 and method_key != 'hermite'):
                fig_single = plt.figure(figsize=(10, 6))
                plot_title_single = f"{method_info['name']}, {node_info['nodes_name']}, n={i}"

                # Plot original function - make linestyle consistent with combined plot
                plot_function(plot_x, func, None, y_lim=y_lim_plot, x_lim=x_lim_plot,
                              function_name='Funkcja Oryginalna', color='black',
                              linestyle=':', # <-- Added linestyle for consistency
                              nodes_name='')

                # Plot interpolation - ADD linestyle from method_info
                plot_function(plot_x, interp_func, nodes=current_nodes,
                              y_lim=y_lim_plot, x_lim=x_lim_plot,
                              function_name=method_info['name'],
                              nodes_name=node_info['nodes_name'],
                              color=method_info['color'],
                              linestyle=method_info['linestyle']) # <-- Added linestyle argument

                plt.title(plot_title_single)
                plt.legend() # Call legend again to ensure all labels are shown

                save_path_single = paths[method_key][node_key]
                plt.savefig(os.path.join(save_path_single, f"{method_info['name']} Węzły {node_info['name']} n={i}.png"))
                plt.close(fig_single) # Close the individual plot figure

                # --- Add to Combined Plot ---
                current_comb_fig = combined_figs[node_key]
                plt.figure(current_comb_fig.number) # Make the correct combined plot active

                # This part already correctly uses color and linestyle
                plot_function(plot_x, interp_func, nodes=None, # No nodes on combined plot lines
                              function_name=f"{method_info['name']}",
                              color=method_info['color'],
                              linestyle=method_info['linestyle'], # Add linestyle
                              nodes_name='',
                              ) # Empty nodes_name

            # --- Record Data --- (Record even if skipped)
            data_records.append({
                'Number of Nodes': i,
                'Node Type': node_info['name'],
                'Interpolation Method': method_info['name'],
                'Max Error': error_max,
                'Scaled RMSE': error_scaled_rmse # Using the name consistent with the function
            })


    # --- Finalize Combined Plots ---

    
    for node_key, fig in combined_figs.items():
        
            
        plt.figure(fig.number) # Activate the figure
        plt.title(f"Porównanie Interpolacji - Węzły {NODE_TYPES[node_key]['name']} (n={i})")
        
        # Adjust legend - place below plot
        num_legend_items = len(INTERP_METHODS) + 1 # +1 for original function
        ncol_legend = min(num_legend_items, 3) # Max 3 columns for legend
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=ncol_legend)
        plt.tight_layout(rect=[0, 0.05, 1, 1]) # Adjust layout to make space for legend
        
        nodes = nodes_all[node_key]
        nodes_name = NODE_TYPES[node_key]['nodes_name']
        plt.scatter(nodes[:, 0], nodes[:, 1], color='red', label=nodes_name, zorder=5) 
        
        plt.savefig(os.path.join(paths['combined'], f"combined_interpolation_nodes_{node_key}_n={i}.png"))
        plt.close(fig) # Close the combined plot figure

# --- Save Data to CSV ---
def format_float(value):
  """Formats floats for CSV, handling inf/nan."""
  try:
    num = float(value)
    if not np.isfinite(num):
      return str(num) # Keep 'inf', '-inf', 'nan' as strings
    # Use scientific notation for very large/small numbers, fixed otherwise
    if abs(num) >= 1e6 or (abs(num) < 1e-4 and num != 0.0):
      return f'{num:.6e}'
    else:
      return f'{num:.6f}'
  except (ValueError, TypeError):
    return value # Return non-numeric values as is

df = pd.DataFrame(data_records)

# Check if DataFrame is not empty before formatting
if not df.empty:
  float_columns = ['Max Error', 'Scaled RMSE']
  for col in float_columns:
    if col in df.columns: # Check if column exists
        df[col] = df[col].apply(format_float)

# Save to CSV
csv_path = os.path.join(PATH_TO_SAVE_DATA, 'interpolation_comparison_errors.csv')
df.to_csv(csv_path, index=False)

print(f"\nProcessing complete. Plots saved in '{PATH_TO_SAVE_MAIN}', CSV saved to '{csv_path}'.")
# Optional: Close any remaining plots if using interactive backend
# plt.close('all') 

Processing n=11 nodes...
Processing n=21 nodes...

Processing complete. Plots saved in '.\img_comparison', CSV saved to '.\data_comparison\interpolation_comparison_errors.csv'.
